<h1>SASSIE ALTO/ALAMO Float Data Use and Visualization</h1>

<p>ALTO/ALAMO Floats are drifting platforms that operate similarly to Argo Floats. 7 floats were deployed durring the campaign. Float data are downloaded and plotted using the code below! <b>Please run the 'Data Download and Metadata Viewing' and 'Supporting Code' sections in order before running the 'Figure Making Code'.</b> </p>
<p>The aim of this notebook is to assist the end user in exploratory data analysis by downloading the SASSIE data from NASA's PODAAC, opening the dataset and displaying it's associated metadata, and creating a few visualizations. This notebook was created by Elizabeth Westbrook. For questions and trouble shooting, please email westbrooke@uncw.edu.</p>

# Data Download and Metadata Viewing

The code in this section will download the dataset from PO.DAAC and open it as an xarray object for metadata and variable attribute viewing.

In [ ]:
import numpy as np
import xarray as xr
import glob 
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import cartopy
import matplotlib
import os
import sys
import pandas as pd
import requests 

<h2>Download The ALTO/ALAMO Float data from PO.DAAC</h2>
<p> All data from the SASSIE campaign is stored on NASA's PO.DAAC. The code in this section of the notebook will download SASSIE ALTO/ALAMO Float data from PO.DAAC, which is accessed through earthdata. If you do not already have an earthdata account, you can create one <a href="https://urs.earthdata.nasa.gov/">here</a> . </p>
    <p>Please enter your earthdata credentials below.</p>

In [ ]:
username = 'your_username'
password = 'your_password'
earthdata = requests.auth.HTTPBasicAuth(username, password)

##LOCAL DIRECTORY TO SAVE FLOAT DATA
dir_in = 'Data/alto_alamo/'

The next block of code creates the directory specified above and downloads the ALTO/ALAMO files to your binder session if it has not already been downloaded.<b> To download the dataset to your local disk from here, right click on the file you want to download and click 'download'. </b>

In [ ]:
##SASSIE DATA DIR
if not os.path.exists(dir_in):
    os.makedirs(dir_in)

float_numbers = ['9101','9097','9098','11131','11132','11133','11136']
for number in float_numbers:
    filename = 'SASSIE_Fall_2022_ALAMO_ALTO_'+number+'.nc'
    if os.path.isfile(dir_in+filename)==False:
        url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_ALTO_ALAMO_FLOATS_V1/'+filename
        with requests.Session() as session:
                session.auth = (username, password)
                r1 = session.request('get', url)
                r = session.get(r1.url, auth=(username, password))
                if r.status_code == 200:
                    if r.ok:
                        with open(dir_in+filename, 'wb') as f:
                            f.write(r.content) 
                            print('Saving Input File: ' + dir_in+filename) 
                else:
                    print("Error:", r.status_code)
                    if r.status_code == 401:
                        print ('Your Username and/or password are incorrect. Please try again')
    else: 
        print('Float '+ number+' file is already in local directory')

<h2> View The Metadata Inside the ALTO/ALAMO Float Files</h2>

The netCDF file has global metadata attributes and attributes associated with each variable. This next block will load data and metadata of the netCDF file into an xarray object (ds). <br> The data set will then be displayed in a clickable HTML format. 


In [ ]:
print('Displaying just 1/7 ALTO/ALAMO files')
#See information about the entire dataset:
files = sorted(glob.glob(dir_in + '/*.nc'))
ds = xr.open_dataset(files[0])
ds

# Supporting Code 
The code in this section provides a set up for the figure making code below by defining directories for data and figures and creating functions that will be called to actually map the data

<h3>Create a Directory to Save Figures</h3>

In [ ]:
#LOCAL DIRECTORY TO SAVE FIGURES
fig_dir ='Figures/alto_alamo/'
#FIGURE DIR 
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

<h3>Define a Colormap and Label for Each Variable in the File</h3>

Within SASSIE's collection of jupyter notebooks, the colormaps used for each variable are held as consistant as possible across all datasets. This function defines the colormap and a label for the variable of interest. 

In [ ]:
#DEFINES COLORMAPS AND LABELS OF EACH VARIABLE IN THIS DATA SET###
def define_variable_attributes(var):
    if var =='salinity':
        colormap = 'viridis'
        var_label = 'Salinity'
    if var =='temperature':
        colormap = 'plasma'
        var_label = 'Water Temperature ($^{\circ}$C)'
    return colormap,var_label

<h3>Define a Function to Create a Map of the Study Area</h3>

The following function creates a map of the SASSIE study area, which is defined by minimum and maximum lat/lon values. These ranges can be changed later when the function is called to zoom in/out on the study area.

In [ ]:
def map_study_area(latmin = 70, latmax =74,lonmin=-157,lonmax=-140):
    
    global fig 
    global ax
    
    #create the map as a figure, set the lat and lon ranges, and add land + river data:
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=cartopy.crs.NorthPolarStereo(central_longitude=-150))
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=cartopy.crs.PlateCarree())
    ax.coastlines(color='k')  
    ax.add_feature(cartopy.feature.LAND, facecolor = '0.50',zorder=1)
    ax.add_feature(cartopy.feature.RIVERS,facecolor='blue')
    #Add lat and lon gridlines and labels:
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3) #draw_labels=True gives lat labels.
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(60,75,1))
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-170,-140,2))
    gl.top_labels = False
    gl.bottom_labels = True 
    gl.right_labels = False
    
    #Add markers for reference cities on the coast:
    if (latmin<71.2906) & (lonmin<-156.7886):
        utqiagvik = ax.scatter(-156.7886,71.2906,s=100,transform=cartopy.crs.PlateCarree(),c='red',marker = '*',label='Utqiagvik, AK',zorder=2)
    if (latmin<70.2002) & (lonmax>-148.4597):
        deadhorse = ax.scatter(-148.4597,70.2002,s=100,c='cyan',transform=cartopy.crs.PlateCarree(),marker = '*',label='Deadhorse, AK',zorder=2)
        

<h3>Define a Function to Create a 3D Grid of Latitude, Longitude, and Depth</h3>

The following function creates a 3D grid for plotting 3 dimensional cast data from the castaway CTD. 

In [ ]:
def make_3d_grid(latmin,latmax,lonmin,lonmax,dmin,dmax):    
        global fig 
        global ax
        
        fig = plt.figure(figsize=(10,8))
        ax = plt.axes(projection='3d',computed_zorder=False)
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False    


        ax.set_xlim(lonmin, lonmax); ax.set_ylim(latmin,latmax); ax.set_zlim(dmin,dmax);
        ax.get_xaxis().get_major_formatter().set_useOffset(False)
        xticks = list(np.linspace(lonmin,lonmax,4))
        ax.set_xticks(np.around(xticks,2))
        ax.set_xticklabels(np.around(xticks,2),rotation=30, ha='center',va='center', minor=False,size='small')

        yticks = list(np.linspace(latmin,latmax,4))
        ax.set_yticks(np.around(yticks,2))
        ax.invert_yaxis()
        ax.set_yticklabels(np.around(yticks,2),rotation=-15,va='bottom', minor=False,size='small')

        zticks = list(np.linspace(dmin,dmax,4))
        ax.set_zticks(np.around(zticks,2))
        ax.set_zticklabels(np.around(zticks,2),ha='center',va='center', minor=False,size='small')

        ax.set_xlabel('Longitude',fontsize=13,labelpad=10,fontweight='bold')
        ax.set_ylabel('Latitude',fontsize=13,labelpad = 7,fontweight='bold')
        ax.set_zlabel('Depth (m)',fontsize=13,labelpad = 16,fontweight='bold')
        ax.view_init(195,280)
        fig.canvas.draw()
        plt.tight_layout()

<h3>Configure Supporting Data to Add to Maps</h3>

<p>The functions for viewing and plotting this data set below have options to include bathymetr and/or shiptrack data to add context to maps. If you are using these options, run
    <br>the following code blocks to:
    <br>1. Create a directory for SASSIE Ship Track data and acess bathymetry data from NOAA
    <br>2. Define functions that add these data to your map when called.</p>

<h4>Create Directory for Shiptrack Data and Access Bathymetry Data </h4>

In [ ]:
## DIRECTORY TO SHIP TRACK DATA
ship_dir =  'Data/TSG/' 

#DOWLOAD SHIPTRACK DATA
if not os.path.isfile(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc'):
    os.makedirs(ship_dir)
    url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_SHIPBOARD_TSG_V1/SASSIE_Fall_2022_Shipboard_TSG.nc'
    with requests.Session() as session:
            session.auth = (username, password)
            r1 = session.request('get', url)
            r = session.get(r1.url, auth=(username, password))
            if r.status_code == 200:
                if r.ok:
                    with open(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc', 'wb') as f:
                        f.write(r.content) 
                        print('Saving Input File: ' + ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc') 
            else:
                print("Error:", r.status_code)
                if r.status_code == 401:
                    print ('Your Username and/or password are incorrect. Please try again')
else: 
    print('Shipboard TSG file is already in local directory')
    
    
#READ IN TOPOGRAPHY/BATHYMETRY DATA
url = 'http://ferret.pmel.noaa.gov/thredds/dodsC/data/PMEL/etopo2.nc'
etopodata = xr.open_dataset(url) 

<h4>Define a Function to Index Relevant Bathymetry Data and Add it to the Map</h4>
This function will index bathymetry data from NOAA within the appropriate spatial range and add it to the map. 


In [ ]:
def add_bathy_data(latmin,latmax,lonmin,lonmax):
        topoin = etopodata.rose.values[0:-1:5,1:-1:5]
        lons = etopodata.etopo2_x.values[0:-1:5]
        lats = etopodata.etopo2_y.values[0:-1:5]
        lons_in_range = lons[np.where((lons >lonmin-1) & (lons<lonmax+1))]
        lats_in_range = lats[np.where((lats >latmin-1) & (lats<latmax+1))]
        topo_in_range = np.squeeze(topoin[np.squeeze(np.where((lats >latmin-1) & (lats<latmax+1))),:][:,np.where((lons >lonmin-1) & (lons<lonmax+1))])
        [bathy_lon,bathy_lat] = np.meshgrid(lons_in_range,lats_in_range)
        
        bathy = ax.contour(bathy_lon,bathy_lat,topo_in_range,np.arange(-6000,-1000,300),transform=cartopy.crs.PlateCarree(),cmap='gray',alpha = 0.2,zorder = 0)

<h4>Define a Function to Add Shiptrack Data to a Map</h4>
This function will pull the lat/lon data from the SASSIE Shipboard TSG file and put it onto a map. 

In [ ]:
def add_ship_track():
        ds_ship = xr.open_dataset(ship_dir+'/SASSIE_Fall_2022_Shipboard_TSG.nc')
        ship_time = np.squeeze(ds_ship['time'])
        ship_lat = np.squeeze(ds_ship['latitude'])
        ship_lon = np.squeeze(ds_ship['longitude'])
       
        track = ax.plot(ship_lon, 
                     ship_lat,linewidth = 0.5,
                     c='black',
                       transform=cartopy.crs.PlateCarree(),label = 'Ship Track',zorder=1)

# Figure Making Code 

<h2>Make A Map of ALTO/ALAMO Float Tracks</h2>
<p>Use the code in this section to plot the ALTO/ALAMO location data on a map colored by time, sea surface temperature, or sea surface salinity. The code blocks in the 'supporting code' section should be run first. The user will need to input a local directory to save figures and ice data for these to work properly. </p>

<h3> Mapping the Time and Location of ALTO/ALAMO Data Collection</h3>

<p>The map_float_paths function shows the tracks of the ALTO/ALAMO Floats throughout the campaign on a map, colored by time. Various features of this function: </p>
    <p>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 
    <br>3. The user has the option to add the SASSIE ship track for reference. 

In [ ]:
#PLOT THE PATHS OF THE FLOATS
def map_float_paths(ship_track=True,bathymetry_data=False):

    #define the lat and lon range for the map:
    latmin = 70
    latmax =74
    lonmin=-157
    lonmax=-144    
    
    #Make the map:
    map_study_area(latmin,latmax,lonmin,lonmax)
    
    #give the map a title:
    ax.set_title('ALTO/ALAMO Float Paths',fontsize=22,pad=1) 

    
    #Add optional map add-ons:
    if ship_track==True:
        add_ship_track()
        
    if bathymetry_data == True:    
        add_bathy_data()
        
    #LOAD AND APPLY FLOAT DATA
    for file in files: 
        ds = xr.open_dataset(file)
        float_time = np.squeeze(ds['time'])
        float_lat = np.squeeze(ds['latitude'])
        float_lon = np.squeeze(ds['longitude'])
        casts = ax.scatter(float_lon,float_lat,s = 10,
                           c = float_time,cmap = 'jet',
                           transform=cartopy.crs.PlateCarree(),zorder=2,
                          vmin=min(xr.open_dataset(files[0]).time.values), vmax=max(xr.open_dataset(files[0]).time.values))
    #make a colorbar
    cbar = fig.colorbar(casts, ax=ax, orientation="horizontal", pad=0.1)
    cbar.set_label(label='Date',size='large',weight='bold')
    cbar_tick_array=(np.linspace(min(xr.open_dataset(files[0]).time.values).astype('int64'),max(xr.open_dataset(files[0]).time.values).astype('int64'),5))
    cbar.set_ticks(cbar_tick_array)
    cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)
    
    ##SAVE FIGURE 
    if not os.path.exists(fig_dir+'map/'):
        os.makedirs(fig_dir+'map/')
    
    print('Saving Output Image:  '+fig_dir+'map/ALTO_ALAMO_paths.png')
    plt.savefig(fig_dir+'map/ALTO_ALAMO_paths.png',dpi='figure',format='png')
    
    


In [ ]:
map_float_paths()

<h2>Three Dimensional Representation of Vertical Profile Data</h2>
Use the code in this section to plot profile locations and depths of a selected ALTO/ALAMO float on a 3D grid, colored by a physical measurement variable from the file (temperature, salinity, density, etc.). The user must select a float based on number. The float numbers were 9097, 9098, 9101, 11131, 11132, 11133, and 11136. 

In [ ]:
##USE TO PLOT THE PROFILES TAKEN BY ANY GIVEN FLOAT ON A 3D GRID##
def float_profiles(float_number,var):
    
    #get float data from the selected float:
    file = dir_in+'/SASSIE_Fall_2022_ALAMO_ALTO_'+str(float_number)+'.nc'
    ds_float = xr.open_dataset(file)
    float_lat = ds_float['latitude']
    float_lon = ds_float['longitude']
    float_depth = ds_float['depth']
    float_var = ds_float[var]
    
    #put lat,lon,and depth in 2D format:
    [lat,depth] = np.meshgrid(float_lat,float_depth)
    [lon,depth] = np.meshgrid(float_lon,float_depth)
    
    
    #assign a colormap and variable label: 
    colormap,var_label = define_variable_attributes(var)

    #define the latitude, longitude, and depth limits of the 3D grid: 
    latmin = min(float_lat.values)
    latmax = max(float_lat.values)
    lonmin = min(float_lon.values)
    lonmax = max(float_lon.values)
    dmin = min(float_depth.values)
    dmax = max(float_depth.values)+0.5
    
    #create the 3D grid 
    make_3d_grid(latmin,latmax,lonmin,lonmax,dmin,dmax)
    
    ax.set_title('ALTO/ALAMO'+' '+var_label+' '+' \n Float '+str(float_number),fontsize=18,y=0.92,x=0.6)

        
    ##APPLY FLOAT DATA   
    a = ax.scatter(lon,lat,depth,c=(float_var.values),cmap = colormap,s=2,zorder=1)    
    
    cbar = fig.colorbar(a,shrink=0.6)
    cbar.set_label(label=var_label,size=15,weight='bold',labelpad=4)
   
    ##ADD BLACK AT TOP OF EACH PROFILE FOR CLARITY
    b = ax.plot(float_lon,float_lat,linewidth=2,
                 color='black',label = 'Float Path',zorder=1)
    
    plt.legend(loc=(0.7,0.3))
    
    ## SAVE THE FIGURE
    if not os.path.exists(fig_dir+var+'/'):
        os.makedirs(fig_dir+var+'/')
    
    print('Saving Output Image:  '+fig_dir+var+'/'+'ALTO_ALAMO_'+var+'_Float_'+str(float_number)+'.png')
    plt.savefig(fig_dir+var+'/'+'ALTO_ALAMO_'+var+'_Float_'+str(float_number)+'.png',dpi='figure',format='png',bbox_inches='tight')
    

In [ ]:
float_profiles(9098,'temperature')
float_profiles(9098,'salinity')

<h2>Plots of Individual ALTO/ALAMO Profiles With a Given Float Number and Time. </h2>
This code creates a plot of the temperature and salinity measurements of one single profile that is closest to a given time and was taken by a given float. Profiles are plotted on the same axis axis in different colors.  

In [ ]:
##PLOT TEMP/SALINITY PROFILE FROM A GIVEN FLOAT CLOSEST TO A GIVEN TIME: 
def plot_float_profile_by_date(float_number,time):
    
    #GET ALL FLOAT DATA
    file = dir_in+'/SASSIE_Fall_2022_ALAMO_ALTO_'+str(float_number)+'.nc'   
    ds_float = xr.open_dataset(file)
    float_time = ds_float['time'].astype('int64')
    
    ##FIND THE CLOSES TIME TO THE TIME SPECIFIED
    time_difference = abs((float_time.values)-np.datetime64(time,'ns').astype('int64'))
    cast_indx = np.squeeze(np.where(time_difference == min(time_difference)))
    
    #PULL OUT LAT,LON,TIME,AND DATA OF CAST
    cast_time = ds_float['time'].values[cast_indx]
    float_lat = ds_float['latitude'].values[cast_indx]
    float_lon = ds_float['longitude'].values[cast_indx]
    float_depth = ds_float['depth'].values
    float_temp = ds_float['temperature'].values[:,cast_indx]
    float_sal = ds_float['salinity'].values[:,cast_indx]
    
    
    ## PLOT THE DATA
    fig,ax1 = plt.subplots()
    ax2 = ax1.twiny()
    ax1.plot(float_temp,float_depth,linewidth=0.4,c='blue')
    ax1.invert_yaxis()
    ax1.set_xlabel('Temperature ($^{\circ}$C)',color = 'blue')
    ax1.set_ylabel('Depth (m)')
    ax2.plot(float_sal,float_depth,linewidth=0.4,c='red')
    ax2.set_xlabel('Salinity',color ='red')
    fig.suptitle('ALTO/ALAMO Float '+str(float_number)+' Salinity and Temperature Profile on '+str(pd.to_datetime(cast_time).year).zfill(2)+'/'+str(pd.to_datetime(cast_time).month).zfill(2)+'/'+str(pd.to_datetime(cast_time).day).zfill(2)+' at '+str(pd.to_datetime(cast_time).hour).zfill(2)+':'+str(pd.to_datetime(cast_time).minute).zfill(2)+':'+str(pd.to_datetime(cast_time).second).zfill(2)+'\n'+str(float_lat)+' $^{\circ}$N, '+str(float_lon)+' $^{\circ}$E',y=1.05)
    
    
    ## SAVE THE FIGURE
    if not os.path.exists(fig_dir+'profiles/'):
        os.makedirs(fig_dir+'profiles/')
    print('Saving Output Image:  '+fig_dir+'profiles/'+'float'+str(float_number)+'_profile_'+str(pd.to_datetime(cast_time).year)+str(pd.to_datetime(cast_time).month).zfill(2)+str(pd.to_datetime(cast_time).day).zfill(2)+str(pd.to_datetime(cast_time).hour).zfill(2)+str(pd.to_datetime(cast_time).minute).zfill(2)+str(pd.to_datetime(cast_time).second).zfill(2)+'.png')
    plt.savefig(fig_dir+'profiles/'+'float'+str(float_number)+'_profile_'+str(pd.to_datetime(cast_time).year)+str(pd.to_datetime(cast_time).month).zfill(2)+str(pd.to_datetime(cast_time).day).zfill(2)+str(pd.to_datetime(cast_time).hour).zfill(2)+str(pd.to_datetime(cast_time).minute).zfill(2)+str(pd.to_datetime(cast_time).second).zfill(2)+'.png',dpi='figure',format='png',bbox_inches='tight')



In [ ]:
plot_float_profile_by_date(9101,'2022-09-21T03:30')
